In [1]:
import pandas as pd
import random
import numpy as np
import math
import copy 

## This code creates a Decision Tree on the Abalone dataset from the UCI Machine Learning Repository https://archive.ics.uci.edu/ml/index.php

[Datasets](#Datasets):   
[Abalone](#Abalone)  

Algorithms-  
[Decision Tree ID3](#Decision_Tree)  
[Reduced Error Pruning](#Pruning)  

[Processing](#Processing) 

<a id="Datasets"></a>
# Datasets

<a id="Abalone"></a>
## Abalone

In [2]:
abalone = pd.read_csv( "abalone.data", header = None,  names = ['sex','length','diameter','height','whole_weight','shucked_weight','viscera_weight','shell_weight','rings'])

In [3]:
abalone.head()

,sex,length,diameter,height,whole_weight,shucked_weight,viscera_weight,shell_weight,rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


# Algorithms

<a id="Decision_Tree"></a>
## Decision Tree- ID3

In [4]:
# Method to create a decision tree and test it on a set of test data
# data is the dataset to be used for the algorithm
# test is a list of indices from data to be used for testing
# validation is a list of indices from data to be used for validation
# train is a list of indices from data to be used for training
# features is a list of the indicies for the features to be used to measure distance
# target_class is the index of the target class feature
# returns a list of classes, the indices which correspond to the indices of test
def test_id3(data, test, validation, train, features, target_class, target_type):
    # get id3 tree
    root = id3(data, test, validation, train, features, target_class, None, None, target_type, None)
    # use tree to get classes for test dataset
    test_classes = []
    for index in range(len(test)):
        # get record
        node = root
        node_feature = node.split_feature # index of feature in data being tested for
        record = data.iloc[[test[index]]]
        check = 0
        record_class = -1
        while(check == 0):
            if(node.is_leaf == True):
                record_class = node.leaf_class
                check = 1
            # end if
            if(node_feature == None):
                record_class = node.leaf_class
                check = 1
            # end if
            # node is not leaf, move to next level
            # cycle through child nodes until find the one we want
            if(check == 0):
                child_nodes = node.children
                check2 = 0
                while(check2 == 0):
                    for child_node_index in range(len(child_nodes)):
                        child_node = child_nodes[child_node_index]
                        # check if node is split on an object feature
                        colname = data.columns[node_feature]
                        if(child_node.feature_quality == None or data[colname].dtype == 'O'):
                            if(child_node.feature_element == record.iloc[0,node_feature]):
                                # this is our child node, update accordingly
                                node = child_node
                                check2 = 1
                                if(child_node.split_feature == None):
                                    node_feature = target_class
                                # end if
                                else:
                                    node_feature = child_node.split_feature
                                # end else  
                            # end if  
                        # end if
                        else:
                            if(child_node.feature_quality > 0 and record.iloc[0,node_feature] >= child_node.feature_element): 
                                # this is our child node, update accordingly
                                node = child_node
                                if(child_node.split_feature == None):
                                    node_feature = target_class
                                # end if
                                else:
                                    node_feature = child_node.split_feature
                                # end else
                                check2 = 1
                            # end if  
                            else:
                                if(child_node.feature_quality < 0 and record.iloc[0,node_feature] < child_node.feature_element):
                                    # this is our child node, update accordingly
                                    node = child_node
                                    if(child_node.split_feature == None):
                                        node_feature = target_class
                                    # end if
                                    else:
                                        node_feature = child_node.split_feature
                                    # end else
                                    check2 = 1
                                # end if
                            # end else
                        # end else
                        
                    # end for
                    if(check2 == 0): # cycled through all child nodes, none of the features matched, return class of node
                        record_class = node.leaf_class
                        check2 = 1
                        check = 1
                    # end if
                # end while               
            # end if
        # end while
        test_classes.append(record_class)
    #end for

    #for index in range(len(test)):
        #print("test class guess:")
        #print(test_classes[index])
        #print("actual:")
        #record = data.iloc[[test[index]]]
        #print(record.iloc[0,target_class])
    return test_classes

In [5]:
# ID3 Decision Tree method
# data is the dataset to be used for the algorithm
# test is a list of indices from data to be used for testing
# validation is a list of indices from data to be used for validation
# train is a list of indices from data to be used for training
# features is a list of the indicies for the features to be used to measure distance
# target_class is the index of the target class feature
# parent node is the parent of the node wanting to be created
# feature_element is the element being tested on with this node
# feature_quality indicates if the split_feature of the parent node is categorical or continous
# returns a list of classes, the indices which correspond to the indices of test
# returns a decision tree
def id3(data, test, validation, train, features, target_class, parent_node, feature_element, target_type, feature_quality):
    # base case, if all train records have the same target class, or only 1 record
    base_record_class = data.iloc[train[0],target_class]
    check = 0
    while(check == 0):
        for index in range(len(train)):
            # get class of record
            record_class = data.iloc[(train[index]),target_class]
            # check if it's a different class
            if(record_class != base_record_class):
                check = 1
            # end if
        #end for
        if(check == 0):
            # all target classes were the same
            leaf = treeNode( None, feature_element, parent_node, None, True, base_record_class, feature_quality)
            return leaf
        # end if
    # end while           
    # second base case, feature set is empty
    if(not features):
        leaf = treeNode(None, feature_element, parent_node, None, True, base_record_class, feature_quality)
        return leaf
    # end if
    # calculate on which feature we should split
    feature_gain_ratios = []
    # get gain ratio for each feature
    for feature_index in range(len(features)):
        feature_gain_ratios.append(gain_ratio(data, train, target_class, features[feature_index]))
    # end for
    # determine best feature to split on
    split_feature = -1
    best_gain = -10000000
    for index in range(len(feature_gain_ratios)):
        if(feature_gain_ratios[index] > best_gain):
            best_gain = feature_gain_ratios[index]
            split_feature = features[index]
        # end if
    # end for
    #calculate class for interior node, max of target class
    unique_classes = []
    class_count = []
    for index in range(len(train)):
        # get record
        record = data.iloc[[train[index]]]
        # get class
        record_class = record.iloc[0,target_class]
        # check if class is already in unique_classes list
        if(record_class in unique_classes):
            # get index of record_class in unique_classes
            class_index = unique_classes.index(record_class)
            # add count to class_count list in corresponding index
            class_count[class_index] = class_count[class_index] + 1
        # end if
        else: # first instance of this class
            unique_classes.append(record_class)
            # get index of record_class in unique_classes
            class_index = unique_classes.index(record_class)
            # add count to class_count list in corresponding index
            class_count.append(1)
        # end else
    # end for
    # end else
    #get class with highest count
    node_class = unique_classes[class_count.index(max(class_count))]
    # create base node to use as parent for children
    node = treeNode(split_feature,feature_element, parent_node, None, False, node_class, feature_quality)
    
    
    # get child nodes
    # determine if split_feature is of object type
    colname = data.columns[split_feature]
    if(data[colname].dtype == 'O'):
        
        new_features = copy.deepcopy(features)
        new_features.remove(split_feature)
        child_nodes = []
        unique_elements = []
        for index in range(len(train)):
            # get record
            record = data.iloc[[train[index]]]
            # get element
            record_element = record.iloc[0,split_feature]
            # check if element is already in unique_elements list
            if(record_element in unique_elements):
                # get index of record_element in unique_elements
                element_index = unique_elements.index(record_element)
            # end if
            else: # first instance of this element
                unique_elements.append(record_element)
                # get index of unique_classes in unique_elements
                element_index = unique_elements.index(record_element)
            # end else
        # end for
        # create a child node for each unique element
        for element in range(len(unique_elements)):
            child_train = []
            child_features = copy.deepcopy(new_features)
            for index in range(len(train)):
                # get record
                record = data.iloc[[train[index]]]
                # see if record has value element
                record_element = record.iloc[0,split_feature]
                if(record_element == unique_elements[element]):
                    child_train.append(train[index])
                # end if
            # end for
            child_nodes.append(id3(data, test, validation, child_train, child_features, target_class, node, unique_elements[element], target_type, None))
        # end for
        node.children = child_nodes
    else:
        new_features = copy.deepcopy(features)
        new_features.remove(split_feature)
        child_nodes = []
        mean = data[colname].mean()
        
        child_train1 = []
        child_train2 = []
        child_features = copy.deepcopy(new_features)
        for index in range(len(train)):
            # get record
            record = data.iloc[[train[index]]]
            # see if record has value element
            record_element = record.iloc[0,split_feature]
            if(record_element >= mean):
                child_train1.append(train[index])
            # end if
            else:
                child_train2.append(train[index])
            # end else
        # end for
        if(len(child_train1)>0):
            child_nodes.append(id3(data, test, validation, child_train1, child_features, target_class, node, mean, target_type, 1))
        # end if
        if(len(child_train2)>0):
            child_nodes.append(id3(data, test, validation, child_train2, child_features, target_class, node, mean, target_type, -1))
        # end if
        node.children = child_nodes
    # end else
    return node

In [6]:
class treeNode:
    
    # initializer
    def __init__(self, split_feature, feature_element, parent, children, is_leaf, leaf_class, feature_quality):
        self.split_feature = split_feature # index of feature on data
        self.feature_element = feature_element # element of parent node feature this split is on
        self.parent = parent
        self.children = children
        self.is_leaf = is_leaf
        self.leaf_class = leaf_class # element of target class to assign
        self.feature_quality = feature_quality # if feature element is the mean of a continous feature, quality indicates if you split on greater than or equal to, or less than

In [7]:
# Method to print the tree
# tree_node is the root of the tree to be printed
# prints to the console, nothing is returned
def print_tree(tree_node):
    print("tree node split feature")
    print(tree_node.split_feature)
    print("tree node feature element")
    print(tree_node.feature_element)

    if(tree_node.is_leaf != True):
        print("children:")
        child_nodes = tree_node.children
        for item in child_nodes:
            print_tree(item)
        # end for
        print("end children for node")
    # end if
    else:
        print("leaf class:")
        print(tree_node.leaf_class)
    # end else
    return

In [8]:
# Method to calculate the entropy
# data is the dataset to be used for the algorithm
# train is a list of indices from data to be used for training
# target_class is the index of the target class feature
# returns the entropy of the data for the training set passed
def entropy(data, train, target_class):
    total_count = len(train)
    unique_classes = []
    class_count = []
    for index in range(total_count):
        # get record
        record = data.iloc[[train[index]]]
        # get class
        record_class = record.iloc[0,target_class]
        # check if class is already in unique_classes list
        if(record_class in unique_classes):
            # get index of record_class in unique_classes
            class_index = unique_classes.index(record_class)
            # add count to class_count list in corresponding index
            class_count[class_index] = class_count[class_index] + 1
        # end if
        else: # first instance of this class
            unique_classes.append(record_class)
            # get index of record_class in unique_classes
            class_index = unique_classes.index(record_class)
            # add count to class_count list in corresponding index
            class_count.append(1)
        # end else
    # end for
    entropy = 0
    for class_index in range(len(unique_classes)):
        entropy = entropy - ((class_count[class_index]/total_count)*math.log((class_count[class_index]/total_count),2))
    # end for
    return entropy

In [9]:
# Method to calculate the expected entropy
# data is the dataset to be used for the algorithm
# train is a list of indices from data to be used for training
# target_class is the index of the target class feature
# feature is the index of the feature being tested for
# returns the expected entropy of a feature for the training set passed
def expected_entropy(data, train, target_class, feature):
    total_count = len(train)
    feature_elements = []
    element_counts = []
    element_indices = []
    colname = data.columns[feature]
    if(data[colname].dtype == 'O'):
        for index in range(total_count):
            # get record
            record = data.iloc[[train[index]]]
            # get element
            record_element = record.iloc[0,feature]
            # check if element is already in feature_elements list
            if(record_element in feature_elements):
                # get index of record_class in feature_elements
                element_index = feature_elements.index(record_element)
                # add count to element_counts list in corresponding index
                element_counts[element_index] = element_counts[element_index] + 1
                # add index of record to element_indices
                element_indices[element_index].append(index)
            # end if
            else: # first instance of this element
                feature_elements.append(record_element)
                # get index of record_element in feature_elements
                element_index = feature_elements.index(record_element)
                # add count to class_count list in corresponding index
                element_counts.append(1)
                # add index of record to element_indices
                element_indices.append([index])
            # end else      
        # end for
    # end if
    else:
        element_counts = [0,0]
        element_indices = [[],[]]
        for index in range(total_count):
            # get mean
            mean = data[colname].mean()
            # get record
            record = data.iloc[[train[index]]]
            # get element
            record_element = record.iloc[0,feature]
            if(record_element >= mean):
                # get index of record_class in feature_elements
                element_index = 0
                # add count to element_counts list in corresponding index
                element_counts[element_index] = element_counts[element_index] + 1
                # add index of record to element_indices
                element_indices[element_index].append(index)
            # end if
            else: 
                # get index of record_element in feature_elements
                element_index = 1
                # add count to class_count list in corresponding index
                element_counts[element_index] = element_counts[element_index] + 1
                # add index of record to element_indices
                element_indices[element_index].append(index)
            # end else    
        # end for
    # end else
    expected_entropy = 0
    for element_index in range(len(feature_elements)):
        expected_entropy = expected_entropy + ((element_counts[element_index]/total_count)*entropy(data, element_indices[element_index],target_class))
    # end for
    return expected_entropy
    

In [10]:
# Method to calcluate the gain ratio
# data is the dataset to be used for the algorithm
# train is a list of indices from data to be used for training
# target_class is the index of the target class feature
# feature is the index of the feature being calculated for
# returns the gain ratio of a feature for the training set passed
def gain_ratio(data, train, target_class, feature):
    gain_ratio = (entropy(data, train, target_class) - expected_entropy(data, train, target_class, feature)) / (info_value(data, train, target_class, feature)+0.00001)
    return gain_ratio

In [11]:
# Method to calcluate the info value
# data is the dataset to be used for the algorithm
# train is a list of indices from data to be used for training
# target_class is the index of the target class feature
# feature is the index of the feature being calculated for
# returns the info value of a feature for the training set passed
def info_value(data, train, target_class, feature):
    total_count = len(train)
    feature_elements = []
    element_counts = []
    colname = data.columns[feature]
    if(data[colname].dtype == 'O'):
        for index in range(total_count):
            # get record
            record = data.iloc[[train[index]]]
            # get element
            record_element = record.iloc[0,feature]
            # check if element is already in feature_elements list
            if(record_element in feature_elements):
                # get index of record_class in feature_elements
                element_index = feature_elements.index(record_element)
                # add count to element_counts list in corresponding index
                element_counts[element_index] = element_counts[element_index] + 1
            # end if
            else: # first instance of this element
                feature_elements.append(record_element)
                # get index of record_element in feature_elements
                element_index = feature_elements.index(record_element)
                # add count to class_count list in corresponding index
                element_counts.append(1)
            # end else      
        # end for
    # end if
    else:
        element_counts = [0,0]
        for index in range(total_count):
            # get mean
            mean = data[colname].mean()
            # get record
            record = data.iloc[[train[index]]]
            # get element
            record_element = record.iloc[0,feature]
            if(record_element >= mean):
                # get index of record_class in feature_elements
                element_index = 0
                # add count to element_counts list in corresponding index
                element_counts[element_index] = element_counts[element_index] + 1
            # end if
            else: 
                # get index of record_element in feature_elements
                element_index = 1
                # add count to class_count list in corresponding index
                element_counts[element_index] = element_counts[element_index] + 1
            # end else    
        # end for
    # end else
    info_value = 0
    for element_index in range(len(feature_elements)):
        info_value = info_value - ((element_counts[element_index]/total_count)*math.log((element_counts[element_index]/total_count),2))
    # end for
    return info_value

In [12]:
# Performance
# data is the dataset to be used for the algorithm
# test is a list of indices from data to be used for testing
# test_classes is a list with the class for the test dataset that includes the estimate for the target_class
# features is a list of the indicies for the features to be used to measure distance
# target_class is the index of the target class feature
# returns the performance of the data passed
def performance(data, test, test_classes, target_class, target_type):
    n_data = len(test) # this is the number of records to be tested
    curr_perf = 0
    # classification error
    for record in range(n_data): # record is index in test 
        test_record = data.iloc[[test[record]]] # pulls record to test
        if(test_record.iloc[0,target_class] != test_classes[record]):
            curr_perf = curr_perf + 1
        # end if
    # end for
    curr_perf = curr_perf / n_data
    return curr_perf

<a id="Pruning"></a>
## Reduced Error Pruning


In [13]:
# Method to create a pruned decision tree and test it on a set of test data
# data is the dataset to be used for the algorithm
# test is a list of indices from data to be used for testing
# validation is a list of indices from data to be used for validation
# train is a list of indices from data to be used for training
# features is a list of the indicies for the features to be used to measure distance
# target_class is the index of the target class feature
# returns a list of classes, the indices which correspond to the indices of test
def test_id3_pruned(data, test, validation, train, features, target_class, target_type):
    #initialize non-pruned tree as best tree and get error
    best_tree = id3(data, test, validation, train, features, target_class, None, None, target_type, None)
    best_perf = test_tree(best_tree, data, validation, train, features, target_class, target_type) # this will be error, so looking to minimize error
    
    # prune tree from leaf up, keep tree if beats performance
    check = 0
    new_tree = copy.deepcopy(best_tree)
    while(check == 0):
        node = new_tree
        while(not node.is_leaf):
            children = node.children
            for child_node in children:
                node = child_node
                if(children is not None):
                    children = children.remove(child_node)
                # end if
            # end for
            if(node.split_feature == None):
                node.is_leaf = True
            # end if
        # end while // node should be a leaf now
        node = node.parent
        node.is_leaf = True
        perf = test_tree(new_tree, data, validation, train, features, target_class, target_type)
        if(perf < best_perf):
            best_tree = new_tree
            best_perf = perf
        # end if
        else:
            check = 1
        # end else
    # end while
    
    # use final tree to get classes for validation dataset
    test_classes = []
    for index in range(len(test)):
        # get record
        node = best_tree
        node_feature = node.split_feature # index of feature in data being tested for
        record = data.iloc[[test[index]]]
        check = 0
        record_class = -1
        while(check == 0):
            if(node.is_leaf == True):
                record_class = node.leaf_class
                check = 1
            # end if
            if(node_feature == None):
                record_class = node.leaf_class
                check = 1
            # end if
            # node is not leaf, move to next level
            # cycle through child nodes until find the one we want
            if(check == 0):
                child_nodes = node.children
                check2 = 0
                while(check2 == 0):
                    for child_node_index in range(len(child_nodes)):
                        child_node = child_nodes[child_node_index]
                        # check if node is split on an object feature
                        colname = data.columns[node_feature]
                        if(child_node.feature_quality == None or data[colname].dtype == 'O'):
                            if(child_node.feature_element == record.iloc[0,node_feature]):
                                # this is our child node, update accordingly
                                node = child_node
                                check2 = 1
                                if(child_node.split_feature == None):
                                    node_feature = target_class
                                # end if
                                else:
                                    node_feature = child_node.split_feature
                                # end else  
                            # end if  
                        # end if
                        else:
                            if(child_node.feature_quality > 0 and record.iloc[0,node_feature] >= child_node.feature_element): 
                                # this is our child node, update accordingly
                                node = child_node
                                if(child_node.split_feature == None):
                                    node_feature = target_class
                                # end if
                                else:
                                    node_feature = child_node.split_feature
                                # end else
                                check2 = 1
                            # end if  
                            else:
                                if(child_node.feature_quality < 0 and record.iloc[0,node_feature] < child_node.feature_element):
                                    # this is our child node, update accordingly
                                    node = child_node
                                    if(child_node.split_feature == None):
                                        node_feature = target_class
                                    # end if
                                    else:
                                        node_feature = child_node.split_feature
                                    # end else
                                    check2 = 1
                                # end if
                            # end else
                        # end else
                        
                    # end for
                    if(check2 == 0): # cycled through all child nodes, none of the features matched, return class of node
                        record_class = node.leaf_class
                        check2 = 1
                        check = 1
                    # end if
                # end while               
            # end if
        # end while
        test_classes.append(record_class)
    #end for
    best_perf = performance(data, validation, test_classes, target_class, target_type) 

    return test_classes

In [14]:
# Method to test the performance of a tree
# tree is the root of the tree to be tested
# data is the dataset to be used for the algorithm
# test is a list of indices from data to be used for testing
# validation is a list of indices from data to be used for validation
# train is a list of indices from data to be used for training
# features is a list of the indicies for the features to be used to measure distance
# target_class is the index of the target class feature
# returns the performance of the tree on the validation set
def test_tree(tree, data, validation, train, features, target_class, target_type):
    # use tree to get classes for validation dataset
    validation_classes = []
    for index in range(len(validation)):
        # get record
        node = tree
        node_feature = node.split_feature # index of feature in data being tested for
        record = data.iloc[[validation[index]]]
        check = 0
        record_class = -1
        while(check == 0):
            if(node.is_leaf == True):
                record_class = node.leaf_class
                check = 1
            # end if
            if(node_feature == None):
                record_class = node.leaf_class
                check = 1
            # end if
            # node is not leaf, move to next level
            # cycle through child nodes until find the one we want
            if(check == 0):
                child_nodes = node.children
                check2 = 0
                while(check2 == 0):
                    for child_node_index in range(len(child_nodes)):
                        child_node = child_nodes[child_node_index]
                        # check if node is split on an object feature
                        colname = data.columns[node_feature]
                        if(child_node.feature_quality == None or data[colname].dtype == 'O'):
                            if(child_node.feature_element == record.iloc[0,node_feature]):
                                # this is our child node, update accordingly
                                node = child_node
                                check2 = 1
                                if(child_node.split_feature == None):
                                    node_feature = target_class
                                # end if
                                else:
                                    node_feature = child_node.split_feature
                                # end else  
                            # end if  
                        # end if
                        else:
                            if(child_node.feature_quality > 0 and record.iloc[0,node_feature] >= child_node.feature_element): 
                                # this is our child node, update accordingly
                                node = child_node
                                if(child_node.split_feature == None):
                                    node_feature = target_class
                                # end if
                                else:
                                    node_feature = child_node.split_feature
                                # end else
                                check2 = 1
                            # end if  
                            else:
                                if(child_node.feature_quality < 0 and record.iloc[0,node_feature] < child_node.feature_element):
                                    # this is our child node, update accordingly
                                    node = child_node
                                    if(child_node.split_feature == None):
                                        node_feature = target_class
                                    # end if
                                    else:
                                        node_feature = child_node.split_feature
                                    # end else
                                    check2 = 1
                                # end if
                            # end else
                        # end else
                        
                    # end for
                    if(check2 == 0): # cycled through all child nodes, none of the features matched, return class of node
                        record_class = node.leaf_class
                        check2 = 1
                        check = 1
                    # end if
                # end while               
            # end if
        # end while
        validation_classes.append(record_class)
    #end for
    perf = performance(data, validation, validation_classes, target_class, target_type) 
    return perf

<a id="Processing"></a>
## Processing

In [15]:
abalone_features = [0,1,2,3,4,5,6,7]
# 10-fold+validation split
n=11
list_of_indices = list(range(len(abalone)))
n_data = len(abalone) #this is the number of records in the data
test_size = int(n_data/n) #this is the number of records in each test set
df = abalone.sample(frac=1).reset_index(drop=True) # a dataframe with shuffled records
df.sort_values(by= df.columns[8]) # sorts the data by the target class
splits = []    
# create n splits 
for i in range(n):
    split = []
    splits.append(split)
# end for
# splitting the folds so that each fold has approx. the same number of each class
counter = 0
for count in range(test_size):
    for i in range(n):
        splits[i].append(n*count +counter)
        counter = counter +1
    # end for
    counter = 0
# end for

test_split = splits[0]
validation_split = splits[1]
train_split = []
for index in range(n_data):
    if(index not in test_split and index not in validation_split):
        train_split.append(index)
    # end if
# end for

In [16]:
# run algorithm
test_classes = test_id3(df, test_split, validation_split, train_split, abalone_features, 8, 2)

In [17]:
output = performance(df, test_split, test_classes, 8, 2) 
print(output)

0.8891820580474934


Pruned

In [18]:
# run algorithm
test_classes2 = test_id3_pruned(df, test_split, validation_split, train_split, abalone_features, 8, 2)

In [19]:
output2 = performance(df, test_split, test_classes2, 8, 2) 
print(output2)

0.8179419525065963
